# Devices and Virtual Devices

In [ ]:
import numpy as np
from dataclasses import replace
from pulser.devices import Device, VirtualDevice, Chadoq2, MockDevice
from pulser.channels import Rydberg, Raman, Microwave
from pulser import Pulse, Sequence, Register

## Physical devices

To perform any computation using Pulser, it is necessary to define a device. Some physical devices are available, they can be accessed via `pulser.devices`. These devices are instances of the `Device` class. They are constrained by physical considerations and all their parameters are defined. Below is presented the specifications of the physical device `Chadoq2`, that are accessible via the `Device.print_specs()` method. 

In [ ]:
# Dipslay Chadoq2's specifications
Chadoq2.print_specs()

## Virtual Devices

### Converting a Device into a Virtual Device

However we sometimes want to perform the computations on a more permissive device, a device that would have more dimensions or more atoms for instance, or more types of channels. This can be done on an emulator behaving like a device. The `VirtualDevice` class is useful to define such an emulator, a virtual device. Let's start by configuring a virtual device having the same parameters as `Chadoq2`. To do this, we use the `Device.to_virtual()` method that creates a virtual device from a physical one. 

In [ ]:
# Converting the Device object in a VirtualDevice object
VirtualChadoq = Chadoq2.to_virtual()
VirtualChadoq.__repr__()

### Changing parameters of a virtual device with `dataclass.replace()`

As proposed earlier, a virtual device gives us the possibility to modify the parameters of a physical device. The function `dataclass.replace()` can be used to create a new `VirtualDevice` having some parameters changed. For example, simulations can be run on a virtual device having the same properties as `Chadoq2` but allowing working in 3 dimensions.

In [ ]:
# Adding a dimension to the emulator
VirtualChadoq3D = replace(VirtualChadoq, dimensions=3)
print(VirtualChadoq3D)

### Unrealistic parameters of a virtual device

Among the parameters defined in a `VirtualDevice`, some are called unrealistic as they do not refer to something physically implementable. For instance, it is possible to change the Rydberg level used in the simulation or to call a channel multiple times in the same sequence. Modifying the Rydberg level has an impact on the coefficient $C_6$ (see [here](https://github.com/pasqal-io/Pulser/blob/develop/pulser-core/pulser/devices/interaction_coefficients/C6_coeffs.json) for the correspondance between the Rydberg level and $C_6$ coefficient).

In [ ]:
# Changing the Rydberg level
VirtualChadoq3D.change_rydberg_level(61)
print(VirtualChadoq3D)

In [ ]:
# Enable the multiple declaration of a channel in a sequence
VirtualChadoq3D = replace(VirtualChadoq3D, reusable_channels=True)
# Creating a square register
reg = Register.square(4, spacing=5)  # 4x4 array with atoms 5 um apart
# Building a sequence with the register and the virtual device
seq = Sequence(reg, VirtualChadoq3D)
# Declare twice the channel "ram_loc"
seq.declare_channel("ch0", "raman_local")
seq.declare_channel("ch1", "raman_local")
# Show the declared channels
print(seq.declared_channels)

### Building your own virtual device

More generally, the `VirtualDevice` class is more permissive than the `Device` class. Actually, some parameters can be left undefined and simpler devices can be built with a `VirtualDevice` instance. A virtual device only needs a `name`, a `dimension` and a `rydberg_level` to be initialized.

In [ ]:
BasicVirtualDevice = VirtualDevice(
    name="BasicMockDevice",
    dimensions=2,
    rydberg_level=61,
)
print(BasicVirtualDevice)

### Defining the channels of your device

Nevertheless, to perform computation, channels have to be defined. The channels enabled on the device are defined in `channel_objects`. Their name can be defined in `channel_id`. If no name is provided, the name of the channels are automatically generated as `channeltype_adressing`. For example, the `Rydberg.Global` channel is automatically named `rydberg_global` in the specifications of `Chadoq2`.

In [ ]:
# This basic device can be used for digital quantum computing
DiQCVirtualDevice = replace(
    BasicVirtualDevice,
    channel_ids=(
        "ryd_loc",
        "ram_loc",
    ),
    channel_objects=(
        Rydberg.Local(None, None, max_duration=None),
        Raman.Local(None, None, max_duration=None),
    ),
)
print(DiQCVirtualDevice)

### An example of virtual device: the `MockDevice`

Actually, there exists a virtual device having all the channels already implemented, with no constraints on the number of atoms, the distance between them. This virtual device is the `MockDevice`.

In [ ]:
MyMockDevice = replace(MockDevice, name="MyMockDevice")
print(MyMockDevice)